<a href="https://colab.research.google.com/github/guifav/curso-ia-aplicada/blob/main/Previsibilidade_de_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
import shap
from google.colab import files, userdata
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML, display
import json
import openai
import io
from datetime import datetime

# Configuração para visualizações
plt.style.use('fivethirtyeight')
sns.set(style='whitegrid')

# Configuração da API da OpenAI
try:
    openai.api_key = userdata.get('OPENAI_API_KEY')
    openai_enabled = True
    print("Chave da API OpenAI carregada com sucesso!")
except Exception as e:
    print(f"Erro ao carregar a chave da API OpenAI: {e}")
    print("O relatório por IA não estará disponível.")
    openai_enabled = False

# Função para upload de dados de treinamento e previsão
def upload_data():
    data_dict = {}

    print("\n==== UPLOAD DE DADOS DE TREINAMENTO ====")
    print("Por favor, faça o upload do arquivo CSV com os dados históricos dos clientes.")
    print("Este arquivo deve conter uma coluna 'Churn' (0 = não cancelou, 1 = cancelou).")

    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'Arquivo "{fn}" foi carregado com sucesso.')
        data_dict['train'] = pd.read_csv(fn)

    print("\n==== UPLOAD DE DADOS PARA PREVISÃO ====")
    print("Agora, faça o upload do arquivo CSV com os novos clientes para previsão.")
    print("Este arquivo deve ter o mesmo formato que o anterior, mas não precisa ter a coluna 'Churn'.")
    print("Se a coluna 'Churn' estiver presente, ela será ignorada para previsão.")

    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'Arquivo "{fn}" foi carregado com sucesso.')
        data_dict['new'] = pd.read_csv(fn)

    return data_dict

# Função para análise exploratória rápida dos dados de treinamento
def analyze_training_data(df_train):
    print("\n==== ANÁLISE RÁPIDA DOS DADOS DE TREINAMENTO ====")

    # Informações gerais
    print(f"Total de registros: {df_train.shape[0]}")
    print(f"Total de características: {df_train.shape[1]}")

    # Taxa de churn
    if 'Churn' in df_train.columns:
        churn_rate = df_train['Churn'].mean() * 100
        print(f"\nTaxa de Churn nos dados de treinamento: {churn_rate:.2f}%")

        # Gráfico da distribuição de churn
        plt.figure(figsize=(8, 5))
        sns.countplot(x='Churn', data=df_train)
        plt.title('Distribuição de Churn nos Dados de Treinamento')
        plt.xlabel('Churn (0 = Não, 1 = Sim)')
        plt.ylabel('Número de Clientes')
        plt.show()

    # Verificando valores ausentes
    missing_values = df_train.isnull().sum().sum()
    if missing_values > 0:
        print(f"\nAtenção: {missing_values} valores ausentes encontrados nos dados de treinamento.")
        missing_cols = df_train.columns[df_train.isnull().any()].tolist()
        print(f"Colunas com valores ausentes: {missing_cols}")

        # Preenchendo valores ausentes
        for col in missing_cols:
            if df_train[col].dtype in ['int64', 'float64']:
                df_train[col].fillna(df_train[col].median(), inplace=True)
            else:
                df_train[col].fillna(df_train[col].mode()[0], inplace=True)
        print("Valores ausentes foram preenchidos automaticamente.")

    return df_train

# Função para pré-processamento dos dados de treinamento e previsão
def preprocess_data(data_dict):
    print("\n==== PRÉ-PROCESSAMENTO DE DADOS ====")

    df_train = data_dict['train']
    df_new = data_dict['new']

    # Verifica se a coluna 'Churn' existe no conjunto de treinamento
    if 'Churn' not in df_train.columns:
        print("ERRO: A coluna 'Churn' não foi encontrada nos dados de treinamento.")
        return None

    # Separando features e target no conjunto de treinamento
    X_train = df_train.drop('Churn', axis=1)
    y_train = df_train['Churn']

    # Para o conjunto de novos dados
    if 'Churn' in df_new.columns:
        print("A coluna 'Churn' foi encontrada nos dados de previsão, mas será ignorada.")
        X_new = df_new.drop('Churn', axis=1)
    else:
        X_new = df_new.copy()

    # Verificando se as colunas são compatíveis
    train_cols = set(X_train.columns)
    new_cols = set(X_new.columns)

    if train_cols != new_cols:
        missing_in_new = train_cols - new_cols
        extra_in_new = new_cols - train_cols

        if missing_in_new:
            print(f"AVISO: As seguintes colunas estão no conjunto de treinamento mas não nos novos dados: {missing_in_new}")
            print("Estas colunas serão preenchidas com valores padrão (0 para numéricas, 'unknown' para categóricas).")

            for col in missing_in_new:
                if df_train[col].dtype in ['int64', 'float64']:
                    X_new[col] = 0
                else:
                    X_new[col] = 'unknown'

        if extra_in_new:
            print(f"AVISO: As seguintes colunas estão nos novos dados mas não no conjunto de treinamento: {extra_in_new}")
            print("Estas colunas serão removidas para a previsão.")

            X_new = X_new.drop(columns=list(extra_in_new))

    # Verificando valores ausentes nos novos dados
    missing_values_new = X_new.isnull().sum().sum()
    if missing_values_new > 0:
        print(f"\nAtenção: {missing_values_new} valores ausentes encontrados nos dados de previsão.")
        missing_cols = X_new.columns[X_new.isnull().any()].tolist()
        print(f"Colunas com valores ausentes: {missing_cols}")

        # Preenchendo valores ausentes
        for col in missing_cols:
            if X_new[col].dtype in ['int64', 'float64']:
                X_new[col].fillna(X_train[col].median(), inplace=True)
            else:
                X_new[col].fillna(X_train[col].mode()[0], inplace=True)
        print("Valores ausentes foram preenchidos automaticamente.")

    # Tratando colunas categóricas com one-hot encoding
    X_train_encoded = pd.get_dummies(X_train, drop_first=True)
    X_new_encoded = pd.get_dummies(X_new, drop_first=True)

    # Garantindo que X_new_encoded tenha as mesmas colunas que X_train_encoded
    for col in X_train_encoded.columns:
        if col not in X_new_encoded.columns:
            X_new_encoded[col] = 0

    # Removendo colunas extras em X_new_encoded
    extra_cols = [col for col in X_new_encoded.columns if col not in X_train_encoded.columns]
    if extra_cols:
        X_new_encoded = X_new_encoded.drop(columns=extra_cols)

    # Reordenando colunas para garantir a mesma ordem
    X_new_encoded = X_new_encoded[X_train_encoded.columns]

    # Normalização dos dados
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_new_scaled = scaler.transform(X_new_encoded)

    print(f"Dimensões do conjunto de treinamento: {X_train_encoded.shape}")
    print(f"Dimensões do conjunto de previsão: {X_new_encoded.shape}")

    return {
        'X_train': X_train_encoded,
        'X_new': X_new_encoded,
        'y_train': y_train,
        'X_train_scaled': X_train_scaled,
        'X_new_scaled': X_new_scaled,
        'feature_names': X_train_encoded.columns,
        'scaler': scaler,
        'original_df_new': df_new  # Mantendo o DataFrame original para referência
    }

# Função para treinamento do modelo e previsão nos novos dados
def train_and_predict(processed_data):
    print("\n==== TREINAMENTO DO MODELO E PREVISÃO ====")

    # Inicialização e treinamento do modelo
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(processed_data['X_train_scaled'], processed_data['y_train'])

    # Previsão nos novos dados
    y_pred_proba = model.predict_proba(processed_data['X_new_scaled'])[:, 1]
    y_pred = (y_pred_proba >= 0.5).astype(int)

    print("Modelo treinado com sucesso e aplicado aos novos dados.")

    # Importância das características
    feature_importance = pd.DataFrame({
        'Feature': processed_data['feature_names'],
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)

    print("\nImportância das características (top 10):")
    print(feature_importance.head(10))

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
    plt.title('Importância das Características')
    plt.tight_layout()
    plt.show()

    return {
        'model': model,
        'feature_importance': feature_importance,
        'y_pred_proba': y_pred_proba,
        'y_pred': y_pred
    }

# Função modificada para explicabilidade do modelo
def model_explainability(model_results, processed_data):
    print("\n==== EXPLICABILIDADE DO MODELO (SHAP) ====")

    model = model_results['model']
    X_new = processed_data['X_new']  # Usamos os novos dados para SHAP
    feature_names = processed_data['feature_names']

    try:
        # Tentativa com feature_perturbation='interventional' que é mais robusto
        explainer = shap.TreeExplainer(model, feature_perturbation='interventional')
        shap_values = explainer.shap_values(X_new)
    except Exception as e:
        print(f"Erro ao criar explicação SHAP detalhada: {e}")
        print("Tentando abordagem alternativa...")

        try:
            # Limitando a amostra para análise SHAP
            sample_size = min(50, X_new.shape[0])
            X_sample = X_new.iloc[:sample_size]

            # Tentar com abordagem mais simples
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_sample)

            print("Usando amostra reduzida para análise SHAP.")
            X_new = X_sample  # Usar a amostra reduzida para o resto da análise
        except Exception as e2:
            print(f"Não foi possível gerar explicação SHAP: {e2}")
            print("Procedendo com análise de importância de características básica.")

            # Usar apenas importância de características do Random Forest
            feature_importance = model_results['feature_importance']

            print("\nImportância das características (Random Forest):")
            print(feature_importance.head(10))

            # Retornar valores vazios
            return None, feature_importance

    # Se chegou aqui, temos valores SHAP para analisar
    # Resumo das características
    try:
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values[1], X_new, feature_names=feature_names)
        plt.title('Resumo de Impacto das Características (SHAP)')
        plt.show()
    except Exception as e:
        print(f"Erro ao criar gráfico de resumo SHAP: {e}")

    # Tentar gráficos de dependência para as características principais
    try:
        # Obter valores SHAP médios para cada característica
        shap_mean_values = np.abs(shap_values[1]).mean(0)
        top_indices = np.argsort(-shap_mean_values)[:3]  # Reduzido para top 3

        # Mostrar gráficos de dependência para as top características
        for idx in top_indices:
            feature = feature_names[idx]
            try:
                plt.figure(figsize=(10, 6))
                shap.dependence_plot(idx, shap_values[1], X_new, feature_names=feature_names)
                plt.title(f'Gráfico de Dependência para {feature}')
                plt.show()
            except Exception as e:
                print(f"Erro ao criar gráfico de dependência para {feature}: {e}")
    except Exception as e:
        print(f"Erro ao calcular importância média SHAP: {e}")

    # Obter valores SHAP médios para cada característica
    try:
        shap_mean_values = np.abs(shap_values[1]).mean(0)
        shap_importance = pd.DataFrame({
            'Feature': feature_names,
            'SHAP_Importance': shap_mean_values
        }).sort_values('SHAP_Importance', ascending=False)

        print("\nImportância SHAP das características (top 10):")
        print(shap_importance.head(10))

        return shap_values, shap_importance
    except Exception as e:
        print(f"Erro ao calcular importância SHAP: {e}")
        # Criar um DataFrame vazio de importância
        shap_importance = pd.DataFrame({
            'Feature': feature_names,
            'SHAP_Importance': np.zeros(len(feature_names))
        })
        return None, shap_importance

# Função para identificar clientes com alto risco de churn
def identify_high_risk(model_results, processed_data, threshold=0.7):
    print(f"\n==== IDENTIFICAÇÃO DE CLIENTES COM ALTO RISCO DE CHURN (Threshold: {threshold}) ====")

    y_pred_proba = model_results['y_pred_proba']

    # Usando novos dados
    df_new = processed_data['original_df_new'].copy()

    # Adicionando as probabilidades e previsões
    df_new['churn_probability'] = y_pred_proba
    df_new['churn_prediction'] = (y_pred_proba >= 0.5).astype(int)

    # Identificando clientes de alto risco
    high_risk = df_new[df_new['churn_probability'] >= threshold].copy()

    print(f"Total de clientes analisados: {len(df_new)}")
    print(f"Total de clientes com probabilidade de churn > 50%: {(df_new['churn_prediction'] == 1).sum()} ({(df_new['churn_prediction'] == 1).mean() * 100:.2f}%)")
    print(f"Total de clientes com alto risco de churn (>= {threshold*100}%): {len(high_risk)}")
    print(f"Porcentagem de clientes com alto risco: {len(high_risk) / len(df_new) * 100:.2f}%")

    if len(high_risk) > 0:
        print("\nAmostra de clientes com alto risco:")
        display(high_risk.head().sort_values('churn_probability', ascending=False))

    # Distribuição de probabilidades
    plt.figure(figsize=(10, 6))
    sns.histplot(df_new['churn_probability'], bins=20, kde=True)
    plt.axvline(x=threshold, color='red', linestyle='--', label=f'Threshold ({threshold})')
    plt.axvline(x=0.5, color='green', linestyle='--', label='Previsão (0.5)')
    plt.title('Distribuição de Probabilidades de Churn')
    plt.xlabel('Probabilidade de Churn')
    plt.ylabel('Número de Clientes')
    plt.legend()
    plt.show()

    # Criando segmentos de risco
    def risk_category(prob):
        if prob >= 0.8:
            return "Muito Alto"
        elif prob >= threshold:
            return "Alto"
        elif prob >= 0.5:
            return "Médio"
        elif prob >= 0.3:
            return "Baixo"
        else:
            return "Muito Baixo"

    df_new['risk_category'] = df_new['churn_probability'].apply(risk_category)

    # Distribuição por categoria de risco
    plt.figure(figsize=(10, 6))
    category_order = ["Muito Baixo", "Baixo", "Médio", "Alto", "Muito Alto"]
    risk_counts = df_new['risk_category'].value_counts().reindex(category_order)
    sns.barplot(x=risk_counts.index, y=risk_counts.values)
    plt.title('Distribuição de Clientes por Categoria de Risco')
    plt.xlabel('Categoria de Risco')
    plt.ylabel('Número de Clientes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return high_risk, df_new

# Função para gerar recomendações personalizadas
def generate_recommendations(high_risk_df, feature_importance, all_data_df):
    print("\n==== RECOMENDAÇÕES PERSONALIZADAS PARA RETENÇÃO ====")

    # Top características que influenciam o churn
    top_features = feature_importance.head(5)['Feature'].values

    print("As principais características que influenciam o churn são:")
    for i, feature in enumerate(top_features, 1):
        print(f"{i}. {feature}")

    # Recomendações gerais baseadas nas características mais importantes
    print("\nRecomendações gerais para reduzir churn:")

    recommendations = []

    for feature in top_features:
        if 'contract' in feature.lower():
            rec = "- Oferecer incentivos para contratos de longo prazo para clientes identificados com alto risco"
            recommendations.append({"feature": feature, "recommendation": rec})
            print(rec)
        elif 'tenure' in feature.lower():
            rec = "- Criar programas de fidelidade baseados no tempo de permanência do cliente"
            recommendations.append({"feature": feature, "recommendation": rec})
            print(rec)
        elif 'monthly' in feature.lower() and 'charges' in feature.lower():
            rec = "- Revisar a política de preços para clientes de alto valor com alta probabilidade de churn"
            recommendations.append({"feature": feature, "recommendation": rec})
            print(rec)
        elif 'service' in feature.lower() or 'support' in feature.lower():
            rec = "- Melhorar a qualidade do suporte técnico e atendimento ao cliente"
            recommendations.append({"feature": feature, "recommendation": rec})
            print(rec)
        elif 'payment' in feature.lower():
            rec = "- Oferecer descontos para pagamentos automáticos"
            recommendations.append({"feature": feature, "recommendation": rec})
            print(rec)
        else:
            rec = f"- Analisar mais profundamente como a característica '{feature}' afeta o churn"
            recommendations.append({"feature": feature, "recommendation": rec})
            print(rec)

    # Analisando segmentos específicos
    print("\nAnálise de segmentos específicos:")

    # Se tivermos categorias de risco, analisar por segmento
    if 'risk_category' in all_data_df.columns:
        high_value_customers = all_data_df[
            (all_data_df['risk_category'].isin(['Alto', 'Muito Alto'])) &
            (all_data_df['churn_probability'] >= 0.7)
        ]

        if len(high_value_customers) > 0:
            print(f"\nIdentificados {len(high_value_customers)} clientes de alto risco que necessitam de atenção imediata")

            # Tentativa de identificar padrões comuns
            print("\nPadrões comuns entre clientes de alto risco:")

            # Para variáveis categóricas (se existirem)
            cat_cols = high_value_customers.select_dtypes(include=['object', 'category']).columns
            for col in cat_cols:
                if col != 'risk_category':
                    value_counts = high_value_customers[col].value_counts()
                    if not value_counts.empty:
                        top_value = value_counts.index[0]
                        percentage = value_counts.iloc[0] / len(high_value_customers) * 100
                        print(f"- {col}: {percentage:.1f}% têm '{top_value}'")

    implementation_steps = [
        "1. Segmentar clientes de alto risco por valor e comportamento",
        "2. Desenvolver ofertas personalizadas baseadas nas características específicas de cada cliente",
        "3. Implementar um sistema de alerta precoce para identificar mudanças no comportamento do cliente",
        "4. Criar um fluxo de trabalho automatizado para disparar ações de retenção",
        "5. Monitorar e avaliar continuamente o sucesso das intervenções de retenção"
    ]

    print("\nPara implementar estas recomendações, considere:")
    for step in implementation_steps:
        print(step)

    return top_features, recommendations, implementation_steps

# Função para exportar resultados
def export_results(all_data_with_proba):
    print("\n==== EXPORTANDO RESULTADOS ====")

    # Criar uma cópia para exportação
    export_df = all_data_with_proba.copy()

    # Formatar as probabilidades para melhor visualização
    export_df['churn_probability'] = export_df['churn_probability'].apply(lambda x: f"{x:.2%}")

    # Preparar para exportação
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"previsao_churn_{timestamp}.csv"

    # Salvar no formato CSV
    export_df.to_csv(filename, index=False)

    print(f"Arquivo de resultados salvo como: {filename}")

    # Download do arquivo
    files.download(filename)

    return filename

# Função para gerar relatório com GPT-4o
def generate_ai_report(data_dict, model_results, feature_importance, shap_importance, high_risk_data, recommendations):
    if not openai_enabled:
        print("Não foi possível gerar o relatório por IA: chave da API OpenAI não configurada.")
        return None

    print("\n==== GERANDO RELATÓRIO PERSONALIZADO COM GPT-4o ====")
    print("Aguarde enquanto o modelo de IA analisa os resultados e gera um relatório completo...")

    # Preparar os dados para enviar para a API
    top_features = feature_importance.head(10).to_dict(orient='records')

    # Usar shap_importance se disponível, caso contrário usar feature_importance
    if shap_importance is not None and not (isinstance(shap_importance, pd.DataFrame) and 'SHAP_Importance' in shap_importance.columns and shap_importance['SHAP_Importance'].sum() > 0):
        top_shap_features = shap_importance.head(10).to_dict(orient='records')
    else:
        # Se não temos valores SHAP válidos, usamos feature_importance
        top_shap_features = feature_importance.rename(columns={'Importance': 'SHAP_Importance'}).head(10).to_dict(orient='records')

    # Estatísticas sobre os dados de previsão
    df_new = data_dict['new']
    prediction_stats = {
        'total_customers': len(df_new),
        'high_risk_customers': len(high_risk_data['high_risk']),
        'high_risk_percentage': len(high_risk_data['high_risk']) / len(df_new) * 100,
        'avg_churn_probability': high_risk_data['all_data']['churn_probability'].mean() * 100,
    }

    # Estatísticas sobre os dados de treinamento
    df_train = data_dict['train']
    training_stats = {
        'total_records': len(df_train),
        'features_count': len(df_train.columns) - 1,  # Excluindo a coluna target
        'churn_rate': df_train['Churn'].mean() * 100
    }

    # Criar um prompt para o GPT-4o
    prompt = f"""
    Você é um consultor especialista em análise de negócios e retenção de clientes. Um sistema de previsão de churn foi executado e gerou os seguintes resultados:

    # DADOS DE TREINAMENTO
    - Total de registros no conjunto de treinamento: {training_stats['total_records']}
    - Número de características usadas: {training_stats['features_count']}
    - Taxa de churn histórica: {training_stats['churn_rate']:.2f}%

    # RESULTADOS DA PREVISÃO
    - Total de clientes analisados: {prediction_stats['total_customers']}
    - Clientes identificados com alto risco de churn: {prediction_stats['high_risk_customers']}
    - Porcentagem de clientes com alto risco: {prediction_stats['high_risk_percentage']:.2f}%
    - Probabilidade média de churn: {prediction_stats['avg_churn_probability']:.2f}%

    # IMPORTÂNCIA DAS CARACTERÍSTICAS (RANDOM FOREST)
    {json.dumps(top_features, indent=2)}

    # IMPORTÂNCIA SHAP (EXPLAINABILITY)
    {json.dumps(top_shap_features, indent=2)}

    # RECOMENDAÇÕES PARA RETENÇÃO
    {json.dumps(recommendations, indent=2)}

    Com base nesses dados, elabore um relatório executivo completo e estruturado que:
    1. Resuma os principais insights da análise
    2. Identifique os principais fatores que contribuem para o churn
    3. Apresente um plano estratégico detalhado para reduzir a taxa de churn
    4. Forneça recomendações específicas para diferentes segmentos de clientes
    5. Sugira métricas de acompanhamento para medir o sucesso das intervenções
    6. Inclua próximos passos e melhorias futuras para o sistema

    O relatório deve ser bem estruturado com títulos e subtítulos, ter tom profissional e ser direcionado para executivos de negócios. Inclua insights que não são óbvios a partir dos dados puros e recomendações que sejam acionáveis.
    """

    try:
        # Chamar a API da OpenAI
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Você é um consultor especialista em análise de dados e retenção de clientes, capaz de transformar dados técnicos em insights acionáveis de negócios."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=4000
        )

        # Extrair o relatório da resposta
        report = response.choices[0].message.content

        # Exibir o relatório
        print("\n" + "="*80)
        print("RELATÓRIO EXECUTIVO GERADO PELA IA")
        print("="*80 + "\n")
        print(report)

        # Salvar o relatório em um arquivo
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"relatorio_churn_{timestamp}.md"

        with open(filename, "w", encoding="utf-8") as f:
            f.write(report)

        print(f"\nRelatório salvo em: {filename}")

        # Download do relatório
        files.download(filename)

        return report

    except Exception as e:
        print(f"Erro ao gerar relatório com GPT-4o: {e}")
        return None

# Função principal
def main():
    print("="*70)
    print("SISTEMA DE PREVISÃO DE CHURN DE CLIENTES - MODO PREVISÃO")
    print("="*70)
    print("\nEste sistema treina um modelo usando dados históricos e faz previsões")
    print("de churn em um novo conjunto de clientes para identificar aqueles com maior risco.")

    # Upload dos dados (treino e previsão)
    data_dict = upload_data()

    # Análise rápida dos dados de treinamento
    data_dict['train'] = analyze_training_data(data_dict['train'])

    # Pré-processamento
    processed_data = preprocess_data(data_dict)

    if processed_data is not None:
        # Treinamento e previsão
        model_results = train_and_predict(processed_data)

        # Explicabilidade do modelo (versão robusta)
        shap_values, shap_importance = model_explainability(model_results, processed_data)

        # Verificar se temos valores SHAP válidos
        if shap_values is None:
            print("Aviso: A análise SHAP completa não foi possível. Usando apenas importância de características do modelo.")
            # Use feature_importance como substituto
            shap_importance = model_results['feature_importance'].rename(columns={'Importance': 'SHAP_Importance'})

        # Identificação de clientes com alto risco
        high_risk, all_data_with_proba = identify_high_risk(model_results, processed_data)

        high_risk_data = {
            'high_risk': high_risk,
            'all_data': all_data_with_proba
        }

        # Geração de recomendações
        top_features, recommendations_list, implementation_steps = generate_recommendations(
            high_risk, model_results['feature_importance'], all_data_with_proba
        )

        # Exportar resultados
        export_filename = export_results(all_data_with_proba)

        # Perguntar se o usuário deseja gerar um relatório com IA
        if openai_enabled:
            generate_report = input("\nDeseja gerar um relatório executivo personalizado usando GPT-4o? (s/n): ").strip().lower()

            if generate_report == 's':
                # Gerar relatório com IA
                ai_report = generate_ai_report(
                    data_dict,
                    model_results,
                    model_results['feature_importance'],
                    shap_importance,
                    high_risk_data,
                    {'recommendations': recommendations_list, 'implementation_steps': implementation_steps}
                )

        print("\n" + "="*70)
        print("CONCLUSÃO")
        print("="*70)
        print("O sistema de previsão de churn foi executado com sucesso.")
        print(f"\nTotal de clientes analisados: {len(all_data_with_proba)}")
        print(f"Clientes com alto risco de churn: {len(high_risk)} ({len(high_risk)/len(all_data_with_proba)*100:.2f}%)")

        print("\nOs resultados detalhados foram exportados para:")
        print(f"- {export_filename}")

        if openai_enabled and 'generate_report' in locals() and generate_report == 's':
            print("- O relatório executivo gerado pela IA também foi salvo e disponibilizado para download")

        print("\nUse estes resultados para implementar estratégias de retenção direcionadas")
        print("aos clientes com maior risco de cancelamento.")
        print("\nObrigado por usar o Sistema de Previsão de Churn!")

# Executar o sistema
if __name__ == "__main__":
    main()